In [ ]:
from api_and_repo_link import link_me_up
import api_and_repo_link
from importlib import reload
reload(api_and_repo_link)
link_me_up()

# old stuff i wrote when i joind OB

In [ ]:

from MAPPINGS import COUNTRY_VALUE_MAP, STATUS_VALUE_MAP, STATUS_VALUE_MAP_CSV, SUBSIDIARY_VALUE_MAP, SUBSIDIARY_VALUE_MAP_CSV, CURRENCY_VALUE_MAP, COLUMN_MAPPING, COLUMN_MAPPING_ADDRESS_BOOK, COLUMN_MAPPING_ADDRESS

import pandas
from oliverbonas_source_dags.apis.api_netsuite import APIConfig_netsuite
from util.zeep_to_df import complete_zeep_df

In [ ]:
from multithread_uploader import upload_all_these_records
api = APIConfig_netsuite()
from util.get_ns_number import get_ns_number

In [ ]:
Customer = api.client.get_type("ns13:Customer")
CustomerSearchBasic = api.client.get_type("ns5:CustomerSearchBasic")
get_ns_number(api, "CustomerSearchBasic")

In [ ]:
def do_value_mappings(old_column_name, customer_dict):
    value = customer_dict[old_column_name]
    # this werid character \xa0 is removed if found...!
    if type(value) == str:
        value = value.replace("\xa0", " ")

    if old_column_name.endswith("_country"):
        return COUNTRY_VALUE_MAP[value]
    elif old_column_name == "currency":
        return CURRENCY_VALUE_MAP[value]
    elif old_column_name == "status":
        return STATUS_VALUE_MAP[STATUS_VALUE_MAP_CSV[value]]
    elif old_column_name == "subsidiary":
        return SUBSIDIARY_VALUE_MAP[SUBSIDIARY_VALUE_MAP_CSV[value]]
    else:
        return value

def to_dict(customer_series):
    customer_series[customer_series.isna()] = None # might work!
    customer_dict = customer_series.to_dict()

    # deal with value mappings here straight away
    for oldCol in customer_dict.keys():
        customer_dict[oldCol] = do_value_mappings(oldCol, customer_dict)

    # now deal with the column mappings
    for oldCol, newCol in COLUMN_MAPPING.items():
        customer_dict[newCol] = customer_dict[oldCol]
        del customer_dict[oldCol]

    addressbookList = {
        "addressbook": [
            {
                "addressbookAddress": dict(),
            },
            {
                "addressbookAddress": dict(),
            },
        ]
    }

    Addbook1 = addressbookList["addressbook"][0]
    Addbook2 = addressbookList["addressbook"][1]
    Address1 = addressbookList["addressbook"][0]["addressbookAddress"]
    Address2 = addressbookList["addressbook"][1]["addressbookAddress"]

    for oldCol, newCol in COLUMN_MAPPING_ADDRESS_BOOK.items():
        Addbook = Addbook1 if oldCol.startswith("Address1_") else Addbook2

        Addbook[newCol] = customer_dict[oldCol]
        del customer_dict[oldCol]

    for oldCol, newCol in COLUMN_MAPPING_ADDRESS.items():
        Address = Address1 if oldCol.startswith("Address1_") else Address2

        Address[newCol] = customer_dict[oldCol]
        del customer_dict[oldCol]

    customer_dict["addressbookList"] = addressbookList
    
    customer_dict["entityId"] = "customer_human_" + str(customer_dict["entityId"])
    customer_dict["externalId"] = "customer_human_" + str(customer_dict["externalId"])

    return customer_dict

In [ ]:
response = api.service_proxy.search(
    CustomerSearchBasic(),
    _soapheaders = api.build_soap_headers()
)
custs = response.body.searchResult.recordList.record

# GDPR so do a lil test

In [ ]:
test_df = pandas.read_csv("test_input_dummy_customers.csv")

In [ ]:
test_list = [Customer(**to_dict(row)) for index, row in test_df.iterrows()]

test_list[0].entityId = "customer !\"£$%^&&*()"
test_list[0].externalId = "customer !\"£$%^&&*()"

test_list[1].entityId = "﷽" * 83
test_list[1].externalId = "wideboy"

test_list[2].entityId = "ꙍ 𝁢 𝋤 ↳ ⁌ 〼 ✛ ⦛ ⺣ 🜹 ꡞ ☰ 𒋾 🯂 ⵙ 𐬑 ᚄ ʮ ⠿ "
test_list[2].externalId = "unicode"

test_list[3].entityId = "😂😂😂😂😂😂😂😂😂😂"
test_list[3].externalId = "lol"


complete_zeep_df(test_list)
upload_all_these_records(test_list, api, "addList")